<div style="color:#D81F26;
           display:fill;
           border-style: solid;
           border-color:#C1C1C1;
           font-size:14px;
           font-family:Calibri;
           background-color:#373737;">
<h2 style="text-align: center;
           padding: 10px;
           color:#FFFFFF;">
======= Playground Dec 2021 =======
</h2>
</div>

# About this notebook

This notebook is for submission to the Playground of Dec 2021. A simple Keras sequential neural network has been usedk. Different configuration of kernel initizlizers and activation functions have been experimented for performance comparison. No. of neurons and layers will be experimented later. 

Some codes are referred and copied from other notebooks.
​
## Summary
* Read the training and testing data sets with memory optimization  
* Perform EDA to examine the data and decide which features can be dropped
* Initialize  a 3-layer Keras sequential neural network with following model configuration
    * Kernel initializer of "lecun_normal" and "he_normal"
    * Activation function of "selu", "relu", "elu" and "swish"
* Get the optimal models for cross-validation and data submission
​

## Reference
* "TPS - Oct 2021 Model with Memory reduced" by S T MOHAMMED @stmohd (URL: https://www.kaggle.com/stmohd/tps-oct-2021-model-with-memory-reduced)
* "TPS - Nov 2021 - NN" by ŞAFAK TÜRKELI @sfktrkl (URL: [https://www.kaggle.com/sfktrkl/tps-nov-2021-nn](https://www.kaggle.com/sfktrkl/tps-nov-2021-nn))

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Basic library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import accuracy_score
from scipy import stats

# Scaler
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Neural Network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

# Cross-Validation
from sklearn.model_selection import StratifiedKFold



# Memory optimization

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')


## from: https://www.kaggle.com/bextuychiev/how-to-work-w-million-row-datasets-like-a-pro
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

df_train = reduce_memory_usage(df_train, verbose=True)
df_test = reduce_memory_usage(df_test, verbose=True)
print('Memory reduced')

In [ ]:
df_train.info()

In [ ]:
#df_train.hist(bins=25, figsize = (20,20))

# Statistics of each variable

In [ ]:
df_train.iloc[:,6:20].describe()

#### From the describe() functions above, the variable Soil_Type7 and Soil_Type15 can be removed due to same value for all instances

# Distribution of Target

In [ ]:
df_train['Cover_Type'].value_counts()

### From the value_counts() function above, there is only 1 instance for class 5.  Thus, treat it as outlier and remove it from the dataset

In [ ]:
#df_train = df_train[df_train['Cover_Type'] != 5]

# Preparation of model data

In [ ]:
# Get train data without the target and ids
X = df_train.iloc[:, 1:-1].copy()
X.drop(columns=['Soil_Type7', 'Soil_Type15'], inplace=True)

# Get the target
le = LabelEncoder()
y = le.fit_transform(df_train['Cover_Type'])

# Create test X, drop ids.
test_X = df_test.iloc[:, 1:].copy()
test_X.drop(columns=['Soil_Type7', 'Soil_Type15'], inplace=True)

In [ ]:
del df_train, df_test

## Training History

The following configuration had been tried.

    # layers.Dense(64, activation='selu')      
    # layers.Dense(64, kernel_initializer='lecun_normal', activation='selu')    
    # layers.Dense(64, kernel_initializer='he_normal', activation='relu')      
    # layers.Dense(64, kernel_initializer='he_normal', activation='elu')
    # layers.Dense(64, activation='swish')

```
Configuration 1: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='selu', BatchNormalization
    70 neurons, activation='selu', BatchNormalization
    40 neurons, activation='selu', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC:  0.9608532499999999
```
```
Configuration 2: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='selu', kernel_initializer='lecun_normal', BatchNormalization
    70 neurons, activation='selu', kernel_initializer='lecun_normal', BatchNormalization
    40 neurons, activation='selu', kernel_initializer='lecun_normal', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC:  0.96171275
```
```
Configuration 3: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='relu', kernel_initializer='he_normal', BatchNormalization
    70 neurons, activation='relu', kernel_initializer='he_normal', BatchNormalization
    40 neurons, activation='relu', kernel_initializer='he_normal', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC: 0.9615895
```
```
Configuration 4: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='swish', BatchNormalization
    70 neurons, activation='swish', BatchNormalization
    40 neurons, activation='swish', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC: 0.9612097500000001
```    
```
Configuration 5: 
    callbacks: EarlyStopping min_delta=0.001, patience=20
    EPOCHS: 50, BATCH_SIZE: 512, N_SPLITS: 5
    150 neurons, activation='elu', kernel_initializer='he_normal', BatchNormalization
    70 neurons, activation='elu', kernel_initializer='he_normal', BatchNormalization
    40 neurons, activation='elu', kernel_initializer='he_normal', BatchNormalization
    7 neurons, activation='softmax'
    
    Overall Mean AUC: 0.96141325
```

---



# Model -  Kears NN

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
early_stopping = callbacks.EarlyStopping(
    monitor="val_acc", 
    min_delta=0.001,           # Minimium amount of change to count as an improvement
    patience=20,               # How many epochs to wait before stopping
    restore_best_weights=True)

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
# Factor by which the learning rate will be reduced        
#    factor=0.1,                
    factor=0.2,                # Factor by which the learning rate will be reduced
    patience=20,                # Number of epochs with no improvement
    min_lr=0.001)              # Lower bound on the learning rate

In [ ]:
# CALLBACKS = [early_stopping]

In [ ]:
# Play with those configurations...
EPOCHS = 200
BATCH_SIZE = 4096
N_SPLITS = 20

In [ ]:
# model.add(tf.keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'))
model = keras.Sequential([
    layers.Dense(600,  activation='selu', kernel_initializer='lecun_normal', input_shape=[X.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(300,  activation='selu', kernel_initializer='lecun_normal'),   
    layers.BatchNormalization(),
    layers.Dense(150,  activation='selu', kernel_initializer='lecun_normal'),    
    layers.BatchNormalization(),
    layers.Dense(75,  activation='selu', kernel_initializer='lecun_normal'),    
    layers.BatchNormalization(),    
    # For a binary classification function use sigmoid
    layers.Dense(7, activation='softmax')])

In [ ]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
# fold = 0
#test_predictions = np.zeros(test_X.shape[0])
skf = StratifiedKFold(n_splits=N_SPLITS, random_state=48, shuffle=True)
#scores = {fold:None for fold in range(skf.n_splits)}
model_list = []
accuracy_list  = []
fold_list  = []
test_predictions = []
for fold,(train_idx, test_idx) in enumerate(skf.split(X, y)):
# for train_idx, test_idx in skf.split(X, y):
    train_X, val_X = X.iloc[train_idx], X.iloc[test_idx]
    train_y, val_y = y[train_idx], y[test_idx]

    # Train
    preproc = StandardScaler() # I tried QuantileTransformer, but StandardScaler seems to be better by 0.005
    train_X = preproc.fit_transform(train_X)
    val_X = preproc.transform(val_X)
    
    # Test
    pred_X = preproc.transform(test_X)

    history = model.fit(
        train_X, train_y,
        validation_data=(val_X, val_y), 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[reduce_lr, early_stopping],        # Put your callbacks in a list
        verbose=0)                  # Turn off training log

    val_y_pred = model.predict(val_X, batch_size=BATCH_SIZE)
    val_y_pred = np.argmax(val_y_pred, axis=1)

    # Evaluation
    accuracy = accuracy_score(val_y, val_y_pred)
    
    accuracy_list.append(accuracy)
    model_list.append(history)
    fold_list.append(fold)
    
    print('Kfold = {}, Acc = {:0.5f}'.format(fold, accuracy))
    
    
    
    pred_y = model.predict(pred_X, batch_size=BATCH_SIZE)
    pred_y = np.argmax(pred_y, axis=1)
    test_predictions.append(pred_y)
    
    
#overall_auc = [np.max(scores[fold]['val_acc']) for fold in range(skf.n_splits)]
print('Overall Mean AUC: ', np.mean(accuracy_list))

# Submission data

In [ ]:
pred_class = le.inverse_transform(stats.mode(test_predictions)[0][0])

In [ ]:
df_submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')
                            
output = pd.concat([df_submission['Id'], pd.DataFrame(pred_class)], axis = 1)
output.columns = ['Id', 'Cover_Type']
output.to_csv('submission.csv', index=False)

print('Done. Ready for submission')